In [1]:
# !pip install minsearch

In [2]:
import requests
import json 

with open('documents.json', 'rt') as f:
    docs_raw = json.load(f)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

print(documents[0])    

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


In [3]:
import minsearch

from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

# index.fit(documents)
# index = minsearch.Index(
#     text_fields = ["question", "section", "course"],
#     keyword_fields = ["course"]
# )
# index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [5]:
question = 'Can I still join the course?'

In [6]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
search_results = search(question)
prompt = build_prompt(question, search_results)

In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-5-nano',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [9]:
answer = llm(prompt)
print(answer)

I can’t determine that from the provided context—the CONTEXT section is empty. Please share the relevant details (e.g., course code, enrollment deadline, or policy) so I can answer accurately.


In [10]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [11]:
rag("How do I patch KDE under FreeBSD?")

'The provided CONTEXT contains no information about patching KDE under FreeBSD. Please share the relevant FAQ entry or more context, and I’ll answer based on that.'

## Agentic RAG

In [12]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [13]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [14]:
prompt = prompt_template.format(question=question, context=context)
answer_json = llm(prompt)
print(answer_json)

{
  "action": "SEARCH",
  "reasoning": "Context is empty. I will search the FAQ database for information about joining the course, including enrollment deadlines, seat availability, prerequisites, and late enrollment policies to determine if the user can still join."
}


In [15]:
answer = json.loads(answer_json)
answer['action']

'SEARCH'

In [16]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [17]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
answer_json = llm(prompt)
print(answer_json)

{
  "action": "SEARCH",
  "reasoning": "User asked if they can still join the course. Since CONTEXT is empty, I'll consult the FAQ for policies on late enrollment, add/drop deadlines, capacity, prerequisites, and how to enroll after classes start. I'll look for information such as eligibility, waitlist options, instructor permission requirements, and registration steps."
}


## Agentic Search

In [18]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result


In [19]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:
{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [20]:
question = 'how do i do well on module 1?'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results = []
previous_actions = []

In [21]:
context=build_context(search_results)

In [22]:
prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions="\n".join(json.dumps(action) for action in previous_actions),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [23]:
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number: 0

In [24]:
answer_json=llm(prompt)
print(answer_json)

{
  "action": "ANSWER",
  "answer": "Here are practical steps to do well on Module 1. These are general but effective across many subjects. If you share the course subject or any specifics (rubric, format, topics), I can tailor this further.\n\n1) Know the objectives and rubric\n- Read the module’s learning outcomes and any grading rubric or note on what counts toward the grade (quizzes, assignments, final assessment).\n- Identify the core topics and the type of questions you’ll face (conceptual, calculations, coding, proofs, etc.).\n\n2) Survey and create a focused outline\n- Skim the module quickly to get the big picture.\n- Create a one-page outline listing key topics, definitions, formulas, and how topics relate to each other.\n- Note any areas that look unfamiliar or challenging for extra attention.\n\n3) Active, structured study\n- For each topic, do active learning: write definitions in your own words, summarize concepts, and create or review flashcards for essential terms.\n- W

In [25]:
answer=json.loads(answer_json)
keywords = answer['keywords']
print(keywords)

KeyError: 'keywords'

In [ ]:
for keyword in keywords:
    search_queries.append(keyword)
    sr = search(keyword)
    search_results.extend(sr)

In [ ]:
previous_actions.append(answer)

In [ ]:
search_results = dedup(search_results)

In [ ]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [ ]:
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number: 2

#### Putting it all together

In [ ]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current ite

In [ ]:
answer

{'action': 'ANSWER',
 'answer': "To be successful in Module 1, focus on alignment with the module's objectives and the grading criteria, and manage your time effectively. Practical steps:\n\n- Review early: Read Module 1’s syllabus, learning objectives, and the assessment rubric to know what counts as a strong submission and how your work will be weighted.\n- Note deadlines: Mark all due dates on a calendar and create a weekly study plan with concrete milestones (reading, notes, drafts, and submission).\n- Do the readings first: Complete required readings/resources before attempting assignments and take concise notes that cover key concepts, definitions, and examples.\n- Plan and draft: Outline tasks before writing or solving problems. Draft submissions with attention to formatting, word limits, citations, and required templates.\n- Use the rubric to self-check: Before submitting, map your work to each rubric criterion to ensure you’ve addressed all elements.\n- Leverage support: Atten

## Functions calling (tool use)

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [ ]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [ ]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [ ]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_hT0sjijfet1bTvOkYYdHDl9S', name='search', type='function_call', id='fc_689f9fa67530819388b89ac7eae392ad01199a5f4ae6b2a8', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"module 1 tips"}', call_id='call_1Y7ASQ7kBM5QcJACHKddDYCT', name='search', type='function_call', id='fc_689f9fa6c2288193a37d7bf3658efa4c01199a5f4ae6b2a8', status='completed')]

In [ ]:
calls = response.output

In [ ]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [ ]:
response = client.responses.create(
    model='gpt-5-nano',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"study tips module 1"}', call_id='call_XNoYr1YsDMMgbQAojwYeZblO', name='search', type='function_call', id='fc_689f9fca900c8193afdf60d013985e6f01199a5f4ae6b2a8', status='completed')]

In [ ]:
for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.text) 

function_call


In [27]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()



In [28]:
chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [29]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-5-nano',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

NameError: name 'tools' is not defined

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [ ]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [ ]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [ ]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

In [ ]:
tools.add_tool(add_entry, add_entry_description)

In [ ]:
tools.get_tools()

In [ ]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()